In [1]:
import numpy as np
import pandas as pd

from sklearn.neural_network import MLPClassifier

from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report
from sklearn.metrics import accuracy_score

# vanilla pipeline
## Data Processing
features: 
- `gameLength` [5, 10]
- `uc` - uncertainty condition: [1, 2, 3]
- `r*` - rewards (unnormalized)
- `c*` - choice: [1, 2]
- `rt*` - response time

predict
- `c5` - the first choice of participants: [1, 2]

In [2]:
df = pd.read_csv("data/allHorizonData_cut.csv")

In [3]:
features = ['gameLength', 'uc', 'r1', 'r2', 'r3', 'r4', 'c1', 'c2', 'c3', 'c4']
target = 'c5'

X = df[features]
y = df[target]
y = y-1 # binary CE only takes in 0, 1

print("--- Feature Data (X) ---")
print(X.head())
print("\n--- Target Data (y) ---")
print(y.head())

--- Feature Data (X) ---
   gameLength  uc  r1  r2  r3  r4  c1  c2  c3  c4
0           5   3  66  80  29  75   2   2   1   2
1          10   3  69  50  51  64   2   2   1   2
2          10   2  31  43  26  36   2   1   2   1
3          10   2  65  77  52  73   1   2   2   1
4          10   2  70  19  43  41   2   1   2   1

--- Target Data (y) ---
0    1
1    0
2    0
3    0
4    1
Name: c5, dtype: int64


In [4]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# normalize per feature
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.fit_transform(X_test)

# sanity check
print(f"X_train {X_train.shape}")
print(f"X_test {X_test.shape}")
print(f"y_train {y_train.shape}")
print(f"y_test {y_test.shape}")

X_train (15360, 10)
X_test (3840, 10)
y_train (15360,)
y_test (3840,)


## modeling

training

In [5]:
mlp = MLPClassifier(
    hidden_layer_sizes=(32, ), 
    max_iter=500, 
    random_state=42,
    early_stopping=True,
    n_iter_no_change=20)

print("Training")
mlp.fit(X_train, y_train)
print("complete")

Training
complete


predicting

In [6]:
y_pred = mlp.predict(X_test)

# results
accuracy = accuracy_score(y_test, y_pred)
print(f"\nModel Accuracy: {accuracy:.2f}")

# report
report = classification_report(y_test, y_pred)
print(f'--- report --- \n{report}')


Model Accuracy: 0.80
--- report --- 
              precision    recall  f1-score   support

           0       0.80      0.79      0.80      1941
           1       0.79      0.80      0.80      1899

    accuracy                           0.80      3840
   macro avg       0.80      0.80      0.80      3840
weighted avg       0.80      0.80      0.80      3840



# experiments

In [34]:
def get_data(features, df):
    target = 'c5'

    X = df[features]
    y = df[target]
    y = y-1 # binary CE only takes in 0, 1

    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

    # normalize per feature
    scaler = StandardScaler()
    X_train = scaler.fit_transform(X_train)
    X_test = scaler.fit_transform(X_test)
    
    return X_train, X_test, y_train, y_test


In [35]:
def experiment(archs, feature_sets, dfs):
    output = []


    for feature in feature_sets:
        print(f'feature: {len(feature)}')


        for arch in archs:
            mlp = MLPClassifier(
                hidden_layer_sizes=arch, 
                max_iter=500, 
                random_state=42,
                early_stopping=True,
                n_iter_no_change=20
            )

            temp = []
            for df in dfs:
                X_train, X_test, y_train, y_test = get_data(feature, df)
                mlp.fit(X_train, y_train)
                
                y_pred = mlp.predict(X_test)
                accuracy = accuracy_score(y_test, y_pred)
                temp.append(accuracy) # 0 = overall, 1 = h1, 2 = h6 (based on `dfs` order)

            output.append({
                'feature': len(feature), 
                'architecture': arch, 
                'accuracy': temp[0],
                'h1': temp[1],
                'h6': temp[2],
                
            })

    return output

In [36]:
feature_base = ['gameLength', 'uc']
feature_r_c = ['gameLength', 'uc', 'r1', 'r2', 'r3', 'r4', 'c1', 'c2', 'c3', 'c4']
feature_r_c_rt = ['gameLength', 'uc', 'r1', 'r2', 'r3', 'r4', 'c1', 'c2', 'c3', 'c4', 'r1', 'r2', 'r3', 'r4']

feature_sets = [feature_base, feature_r_c, feature_r_c_rt]

In [37]:
df_h1 = df[df['gameLength'] == 5].copy()
df_h6 = df[df['gameLength'] == 10].copy()

df_set = [df, df_h1, df_h6]

In [38]:
archs_2_layers = [
    (8, ),
    (16, ),
    (32, ),
    (64, ),
    (128, ),
    (256, ),
]

archs_3_layers = [
    (256, 128), 
    (128, 64), 
    (64, 32), 
    (32, 16), 
    (16, 8), 
]

In [39]:
print("--- running 2-layer MLP experiment ---")
arch_2_output = experiment(archs_2_layers, feature_sets, df_set)

print("--- running 3-layer MLP experiment ---")
arch_3_output = experiment(archs_3_layers, feature_sets, df_set)


--- running 2-layer MLP experiment ---
feature: 2
feature: 10
feature: 14
--- running 3-layer MLP experiment ---
feature: 2
feature: 10
feature: 14


In [41]:
arch_2_df = pd.DataFrame(arch_2_output)
arch_3_df = pd.DataFrame(arch_3_output)

best_model_2 = arch_2_df.sort_values(by='accuracy', ascending=False)
best_model_3 = arch_3_df.sort_values(by='accuracy', ascending=False)

print("--- 2 Layer Best ---")
print(best_model_2.head())
print("--- 3 Layer Best ---")
print(best_model_3.head())


worst_model_2 = arch_2_df.sort_values(by='accuracy', ascending=True)
worst_model_3 = arch_3_df.sort_values(by='accuracy', ascending=True)

print("--- 2 Layer Worst ---")
print(worst_model_2.head())
print("--- 3 Layer Worst ---")
print(worst_model_3.head())

--- 2 Layer Best ---
    feature architecture  accuracy        h1        h6
11       10       (256,)  0.799740  0.846875  0.729167
17       14       (256,)  0.799479  0.842708  0.729688
14       14        (32,)  0.798438  0.844271  0.720833
8        10        (32,)  0.797135  0.845833  0.717708
7        10        (16,)  0.796615  0.842708  0.727083
--- 3 Layer Best ---
    feature architecture  accuracy        h1        h6
13       14     (32, 16)  0.804167  0.847917  0.722396
10       14   (256, 128)  0.802604  0.845313  0.723437
8        10     (32, 16)  0.802083  0.849479  0.728125
11       14    (128, 64)  0.801823  0.847917  0.727083
9        10      (16, 8)  0.800781  0.842708  0.716667
--- 2 Layer Worst ---
   feature architecture  accuracy        h1        h6
2        2        (32,)  0.508854  0.492708  0.534896
3        2        (64,)  0.516406  0.522917  0.534896
4        2       (128,)  0.516406  0.522917  0.541146
1        2        (16,)  0.525000  0.492708  0.541146
0     